# Create images DataFrame

In [ ]:
# CREATE IMAGES DATAFRAME FROM ORDERS DATAFRAME AND DATA FROM ETI - BY IMAGE LIST

images_df = pd.DataFrame()
images_df["Order ID"] = orders_df["Order ID"]
images_df["images_list"] = orders_df["Order ID"].apply(env.eti_api.get_image_list_by_orderid)

images_df = pd.concat([orders_df.set_index('Order ID'),
                       images_df.set_index('Order ID')], axis=1, join='inner').reset_index()

images_df = images_df.explode('images_list').rename(columns={'images_list': 'Image_ID'}).reset_index(drop=True)

# images_data = images_df["Image_ID"].apply(env.eti_api.get_image_list_by_orderid)


In [ ]:
# ADD IMAGES DATA BY ORDER ID FROM ETI (without splitting list of orders)

import tqdm.notebook as tq
import warnings
example_images_df = env.eti_api.get_images_data_by_orderid(orders_list[0])['images']
images_df = pd.DataFrame(columns=example_images_df[0].keys())
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    for order in tq.tqdm(orders_list, leave=False):
        order_df = pd.DataFrame(env.eti_api.get_images_data_by_orderid(order)['images'])
        images_df = pd.concat([images_df, order_df], axis='rows', ignore_index=True)

# Data exploration

In [ ]:
env.eti_api.get_categories_hierarchy()

In [ ]:
# SHOW RANDOM IMAGE FROM THE DATAFRAME

example_image_id = random.sample(list(top_weeds_df['imageID']), 1)[0]
image_data = top_weeds_df[top_weeds_df['imageID'] == example_image_id].reset_index()
image_num_tags = image_data.at[0, 'num_weed_tags']
image_crop_name = image_data.at[0, 'cropName']

im_path = env.download_image(example_image_id)
image = io.imread(im_path)

plt.imshow(image)
plt.title(f"Image ID: {example_image_id}\nCrop Type: {image_crop_name}, Weed Tags: {image_num_tags}")
plt.show()

In [ ]:
wide_im_path = env.download_image(matching_wide_image_id)
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# ADD WIDE IMAGE_ID TO EACH ZOOM IMAGE
example_wide_image = env.eti_api.get_matching_wide_images([7275584])
# wide_images = env.eti_api.get_matching_wide_images(list(top_weeds_df['imageID']))
wide_im_path = env.download_image(example_wide_image[0])
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# ADD WIDE IMAGE_ID TO EACH ZOOM IMAGE
example_wide_image = env.eti_api.get_matching_wide_images([7275584])
# wide_images = env.eti_api.get_matching_wide_images(list(top_weeds_df['imageID']))
wide_im_path = env.download_image(example_wide_image[0])
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# VIEW AND FIND INFORMATION ABOUT NUMBER OF WEED TAGS IN THE IMAGES
images_with_weeds_df = images_df[images_df['num_weed_tags']]
print(len(images_with_weeds_df))
plt.hist(images_with_weeds_df['num_weed_tags'], bins=100)
plt.show()

In [ ]:
plt.hist(filtered_images_df['cameraAngle'], bins=100)
plt.show()

In [ ]:
plt.hist(images_df['cameraAngle'], bins=100)
plt.show()

In [ ]:
# images_with_the_most_weeds_tags['num_weed_tags']
plt.hist(top_weeds_soy['num_weed_tags'], bins=100)
plt.show()

In [ ]:
images_df.columns

In [ ]:
example_orderid = random.sample(orders_list, 1)[0]

im_list = env.eti_api.get_image_list_by_orderid(example_orderid, [2])
example_image_id = random.sample(im_list, 1)[0]

im_path = env.download_image(example_image_id)
image = io.imread(im_path)

plt.imshow(image)
plt.show()

In [ ]:
categories_hierarchy = env.eti_api.get_categories_hierarchy()


In [ ]:
for i in range(len(categories_hierarchy)):
    print(f"{i}, category: {categories_hierarchy[i]['id']}")

In [ ]:
# ADD CATEGORIES NAME AND TYPES TO IMAGES DATAFRAME

def create_cat_dict(cat_list):
    cat_dict_types = {}
    cat_dict_names = {}
    for l in cat_list:
        if 'type' in l:
            cat_dict_types[l['id']] = l['type']
        else:
            cat_dict_types[l['id']] = 'NoType'
        if 'name' in l:
            cat_dict_names[l['id']] = l['name']
        else:
            cat_dict_names[l['id']] = 'NoName'
    return cat_dict_types, cat_dict_names



def get_weed_subcategory_name_by_subcategory_id(types_ids_list):
    types_ids_list = eval(types_ids_list)
    print(types_ids_list)
    cat_dict_types, cat_dict_names = create_cat_dict(categories_hierarchy[9]['subCategories'])
    names_list = []
    types_list = []
    print(len(types_ids_list))
    for i in range(len(types_ids_list)):
        print(cat_dict_names[types_ids_list[i]])
        names_list.append(cat_dict_names[types_ids_list[i]])
        types_list.append(cat_dict_types[types_ids_list[i]])

    
    
    # weed_subcategories = categories_hierarchy[9]['subCategories']
    # names_list = []
    # for i in range(len(types_ids_list)):
    #     for j in range(len(weed_subcategories)):
    #         if weed_subcategories[j]['id'] == types_ids_list[i]:
    #             names_list.append(weed_subcategories[j]['name'])

    # if len(names_list) != len(types_ids_list):
        # raise Exception("names_list and types_ids_list lengths are different")
    return names_list

images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)]
# print(images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)])
# images_df['weed_types_list'] = images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)]

In [ ]:
images_df = pd.read_csv("/mnt/disks/datasets/wide_images/images_df_new_1.csv")
unique_weed_types = np.unique(images_df['weed_types_dict'].explode(), return_counts=True)

In [ ]:
# EXCLUDED IMAGES FOR CREATING FINAL_DF:
# FROM interesting_weed_df: [8980681, 8980665, 7076176, 8980677, 7073428, 8980673, 7077207, 7073971, 8980669]
# FROM small_num_of_tags_df_soy : [8190898, 10384238, 6513168, 9182595, 8016877, 8415772, 6767859, 8733629, 9120611, 8730740, 6467322]
# FROM small_num_of_tags_df_corn :
# excluded_image_ids.extend([8980681, 8980665, 7076176, 8980677, 7073428, 8980673, 7077207, 7073971, 8980669])
# excluded_image_ids.extend([8190898, 10384238, 6513168, 9182595, 8016877, 8415772, 6767859, 8733629, 9120611, 8730740, 6467322])
# excluded_image_ids.extend([7037805, 6245305, 7836752, 7836449, 7899418, 7836202, 7043002])
# excluded_image_ids.extend([6325835, 6325719, 6325835])
# excluded_image_ids.extend([5729837, 8724366, 5729837, 5828866, 7652376])
# excluded_image_ids.extend([5682996, 5397485, 8718398, 5396859, 8718440, 5161751])
# excluded_image_ids.extend([5682996, 5161751, 6264631, 8718440, 8718398])


In [ ]:
# CHECK RAM USAGE

import psutil

# Get the system's memory information
memory_info = psutil.virtual_memory()
# Access the available RAM in bytes
available_ram = memory_info.available
# Convert bytes to gigabytes (GB)
available_ram_gb = available_ram / (1024 ** 3)

# Get the memory usage for the current process (kernel)
process = psutil.Process()
memory_info = process.memory_info()
memory_usage_mb = memory_info.rss / 1024 / 1024
memory_usage_gb = memory_usage_mb / 1024

print(f"Memory usage: {memory_usage_gb} GB")

print(f"Available RAM: {available_ram_gb} GB")

In [ ]:
# EXAMPLE - ADDING ORDERS DATA TO IMAGES DF

orders_df = pd.DataFrame({'orders' : [1,2,3,4], 'pass_num' : [1,1,2,2]})
images_df = pd.DataFrame({'orders' : [1, 1, 1, 2, 2, 3, 4, 4, 4], 'im_num' : [111, 121, 122, 123, 333, 444,555, 333, 999]})

merged_df = images_df.merge(orders_df, on='orders')

# from late flights data exploration

In [ ]:
# FILTER IMAGES DATAFRAME
from datetime import datetime

im_df = im_df[(im_df['cameraAngle'] > -95) & (im_df['cameraAngle'] < -85)]
im_df = im_df[im_df['deleted'] != True]

# im_df['weed_types_ids_list'] = im_df['weed_types_ids_list'].apply(eval)
# im_df['uploadDate_time'] = im_df['uploadDate'].apply(datetime.fromtimestamp)
# im_df['uploadDate_date'] = im_df['uploadDate_time'].apply(lambda x: x.to_pydatetime().date())

# im_df.columns
len(im_df)

In [ ]:
import numpy as np
im_df['pass_num_new'] = None
im_df['pass_num_new'] = np.where(im_df['fieldID'].isin(early_flights_field_ids), 1, im_df['pass_num_new'])

In [ ]:
# SHOW IMAGES FROM THE DATAFRAME
from skimage import io
import matplotlib.pyplot as plt


# df_to_show = df_for_resolution_lim_checkup[20:].reset_index(drop=True)
df_to_show = example_images.reset_index(drop=True)

for i in range(len(df_to_show)):
    example_image_id = df_to_show['imageID'][i]
    # SHOW IMAGES
    print(f"Zoom image id: {example_image_id}")
    image_data = df_to_show[df_to_show['imageID'] == example_image_id].reset_index(drop=True)

    # # image_data = df_to_show[df_to_show['imageID'] == example_image_id]
    # # matching_wide_image_id = int(image_data.at[0, 'wideImageID'])
    # # print(f"Wide image id: {matching_wide_image_id}")
    # image_num_tags = image_data.at[0, 'num_weed_tags']
    # image_num_manual_tags = image_data.at[0, 'weed_manualTagsCount']
    # image_num_inference_tags = image_data.at[0, 'weed_inferenceTagsCount']
    pass_num = image_data.at[0, 'pass_num']
    # image_crop_name = image_data.at[0, 'cropName']
    # upload_date = image_data.at[0, 'uploadDate_date']
    # weed_types = image_data.at[0, 'weed_types_ids_list']
    # # weed_types = eval(image_data.at[0, 'weed_types_ids_list'])
    # weed_types = list(map(cat_dict_names.get, weed_types))


    im_path = env.download_image(int(example_image_id))
    image = io.imread(im_path)

    # wide_im_path = env.download_image(int(matching_wide_image_id))
    # wide_image = io.imread(wide_im_path)

    fig, axes = plt.subplots(nrows=1, ncols=1)
    axes.imshow(image, extent=[0, image.shape[1], 0, image.shape[0]])
    # axes[1].imshow(wide_image, extent=[0, wide_image.shape[1], 0, wide_image.shape[0]])


    axes.set_xticks([])
    axes.set_yticks([])
    fig.set_size_inches(20, 10)
    plt.suptitle(f"pass num: {pass_num}")
    # plt.suptitle(f"Crop Type: {image_crop_name}\nZoom Image ID: {example_image_id}\nuploadDate: {upload_date}\nNum Weed Tags: {image_num_tags}\nManual tags: {image_num_manual_tags}, Inference tags: {image_num_inference_tags}\nWeed types: {weed_types}")
    # plt.suptitle(f"Crop Type: {image_crop_name}\nZoom Image ID: {example_image_id}, Wide Image ID: {matching_wide_image_id}\nuploadDate: {upload_date}\nNum Weed Tags: {image_num_tags}\nManual tags: {image_num_manual_tags}, Inference tags: {image_num_inference_tags}\nWeed types: {weed_types}")

    plt.tight_layout()
    plt.show()


In [ ]:
im_df['pass_num'] = [int(eval(im_df['Pass #'][i])['value']) for i in range(len(im_df))]

# Create grid json

In [ ]:
# example_json_path = "data/dataloop_jsons/6415975.json"
example_json_path = 'data/dataloop_jsons/example_json_from_itayg.json'
with open(example_json_path) as file:
    example_data = json.load(file)

# example_polygon_coordinates = example_data['annotations'][0]['coordinates']

In [ ]:
def add_random_boxes_coords(items_df, image_shape, num_boxes=10, grid_shape=(507, 434)):
    grid_coords = get_grid_coords(image_shape)


    random_boxes = grid_coords.sample(n=10)

    top_left_random_boxes = np.random.choice(top_left_list, size=num_boxes, replace=False)

    bottom_right_random_boxes = top_left_random_boxes + np.array(grid_shape)

    items_df['top_left'] = np.array(list(map(tuple, top_left_random_boxes)))
    items_df['bottom_right'] = np.array(list(map(tuple, bottom_right_random_boxes)))

    # np.array(list(zip(top_random_selection, left_random_selection)))
    items_df['boxes_coords'] = {'top_left': np.array(list(map(tuple, top_left_random_boxes))),
                                'bottom_right': np.array(list(map(tuple, bottom_right_random_boxes)))}
    return items_df

In [ ]:
    # example_image_id_from_dataset = items_df['name'][0]
    # example_im_path = env.download_image(int(example_image_id_from_dataset))
    # example_image = io.imread(example_im_path)

In [ ]:
# ADD TO DATAFRAME IMAGES FROM TTT
ttt_images_paths = glob.glob(os.path.join("images_to_dataloop_ttt/wide_ttt", "*.jpg"))
ttt_images_ids = [int(os.path.basename(p).replace(".jpg", "")) for p in ttt_images_paths]
df_existing_images = weeds_df[weeds_df['imageID'].isin(ttt_images_ids)]

In [ ]:
# ZOOM IMAGES THAT DOESNT HAVE WIDE 

problematic_images_zoom = images_data_new[images_data_new['wideImageID'].isna()]['imageID']
wide_checkup = env.eti_api.get_matching_wide_images(list(problematic_images_zoom))

# Resolution reduction

In [ ]:
# BLUR IMAGE - BLOCK REDUCE

resolution_options = [7, 8, 9, 10]  # mm / pixel


random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

fig, axs = plt.subplots(2, 2, figsize=(24, 8), tight_layout=True, constrained_layout=True)
axs = axs.flatten()
for i, ax in enumerate(axs):
    processed_image = skimage.measure.block_reduce(original_image, (resolution_options[i],resolution_options[i],1), np.mean).astype(np.uint8)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    ax.imshow(processed_image)
    ax.set_title(f"Block size: {resolution_options[i]}")
    # del upscaled_image

# plt.tight_layout()
plt.show()

In [ ]:
# RESOLUTION CALCULATION - TRY. NOT GOOD


def get_x_y_px(resolution, new_resolution, fp_x_m, fp_y_m):
    x_px = int(resolution/new_resolution) * fp_x_m
    y_px = int(resolution/new_resolution) * fp_y_m
    return x_px, y_px


def get_image_resolution(height, focal_length):
    fp_x_m = (height / focal_length) * CCD_X # m
    fp_y_m = (height / focal_length) * CCD_Y # m
    # total_ground_area = fp_x_m * fp_y_m # m^2
    resolution = 0.5*(fp_x_m/IMAGE_X_PX + fp_y_m/IMAGE_Y_PX)*1000 # mm/px
    return resolution, fp_x_m, fp_y_m


# random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
random_image_id = 9568380
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

flights_heights_options = np.arange(20, 55, 5)

for h in flights_heights_options:
    focal_length = random_image_data['focalLength'].values[0]
    flight_height = random_image_data['heightAboveGround'].values[0]
    image_resolution, fp_x_m, fp_y_m = get_image_resolution(flight_height, focal_length)
    new_image_resolution, new_fp_x_m, new_fp_y_m = get_image_resolution(h, focal_length)
    print(f"image_resolution: {image_resolution:.2f}, image_new_resolution:{new_image_resolution:.2f}")
    print(f"new_fp_x_m: {new_fp_x_m:.2f},new_fp_y_m: {new_fp_y_m:.2f}")
    image_x_px_new, image_y_px_new = get_x_y_px(image_resolution, new_image_resolution)
    print(f"new image shape: ({image_x_px_new, image_y_px_new})")



In [ ]:
# SHOW IMAGES DIFFERENT RESOLUTIONS

random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

num_images = len(resolutions_dict)
num_rows = (num_images + 1) // 2  # Calculate the number of rows needed

fig, axs = plt.subplots(num_rows, 2, figsize=(8, 8))  # Create subplots

for i, height in enumerate(resolutions_dict.keys()):
    processed_image = resize(original_image, (resolutions_dict[height][0], resolutions_dict[height][0], 3), anti_aliasing=True)
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    ax.imshow(processed_image)
    ax.axis('off')
    ax.set_title(f"Height: {height}")

# If the number of images is odd, remove the last subplot
if num_images % 2 != 0:
    axs[num_rows - 1, 1].axis('off')
    fig.delaxes(axs[num_rows - 1, 1])

plt.tight_layout()
plt.show()

In [ ]:
# BLUR IMAGE BY RESOLUTION

random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

fig, axs = plt.subplots(2, 2, figsize=(24, 8), tight_layout=True, constrained_layout=True)
axs = axs.flatten()
for i, ax in enumerate(axs):
    processed_image = resize(original_image, (resolutions_dict[50][0], resolutions_dict[50][0], 3), anti_aliasing=True)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    # processed_image = skimage.measure.block_reduce(original_image, (resolution_options[i],resolution_options[i],1), np.mean).astype(np.uint8)
    # processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    ax.imshow(processed_image)
    ax.set_title(f"Block size: {resolution_options[i]}")
    plt.show()


# plt.tight_layout()
plt.show()

In [ ]:
# DISPLAY ONE POLYGON ON DIFFERENT RESOLUTION IMAGES

example_image_id = random_image_id
# example_image_id = 7843185
# example_image_id = 6415975
CHOSEN_TAG = 1

# example_json_path = "data/dataloop_jsons/6415975.json"
example_json_path = f"data/dataloop/annotations_anafa_2023_06_23_resolution_lim_dataset_1_v0/json/{example_image_id}.json"
images_resolutions_paths_list = glob.glob(os.path.join(resized_images_folder, f"{example_image_id}*.jpg"))
images_resolutions_paths_list = [images_resolutions_paths_list[0], images_resolutions_paths_list[-1]]


with open(example_json_path) as file:
    example_data = json.load(file)

example_polygon_coordinates = example_data['annotations'][0]['coordinates']
# im_path = env.download_image(int(example_image_id))

x_values = [coord['x'] for coord in example_data['annotations'][CHOSEN_TAG]['coordinates'][0]]
y_values = [coord['y'] for coord in example_data['annotations'][CHOSEN_TAG]['coordinates'][0]]
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

num_images = len(images_resolutions_paths_list)
num_rows = num_images // 2  # Divide and round up
num_cols = min(num_images, 2)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 10))

axs = axs.flatten()
for i, ax in enumerate(axs):

    image = io.imread(images_resolutions_paths_list[i])
    polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')

    ax.imshow(image)
    ax.add_patch(polygon)

    ax.set_xlim(min(x_values)-100, max(x_values)+100)
    ax.set_ylim(min(y_values)-100, max(y_values)+100)

    subtitle = images_resolutions_paths_list[i].replace(f"{example_image_id}_", "")
    ax.set_title(f"image: {os.path.basename(subtitle)}")

fig.suptitle(f"Image ID: {example_image_id}", y=0.8)
fig.tight_layout()

plt.show()

In [ ]:
# DISPLAY ALL DL POLYGONS ON THE IMAGE

# example_image_id = 6044767
example_image_id = 6415975

# example_json_path = "data/dataloop_jsons/6415975.json"
example_json_path = f"data/dataloop/annotations_anafa_2023_06_23_resolution_lim_dataset_1_v0/json/{example_image_id}.json"

with open(example_json_path) as file:
    example_data = json.load(file)

example_polygon_coordinates = example_data['annotations'][0]['coordinates']
im_path = env.download_image(int(example_image_id))
image = io.imread(im_path)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

num_images = len(example_data['annotations'])
num_rows = num_images // 2  # Divide and round up
num_cols = min(num_images, 2)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 10))

axs = axs.flatten()
for i, ax in enumerate(axs):

    x_values = [coord['x'] for coord in example_data['annotations'][i]['coordinates'][0]]
    y_values = [coord['y'] for coord in example_data['annotations'][i]['coordinates'][0]]
    polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')

    ax.imshow(image)
    ax.add_patch(polygon)

    ax.set_xlim(min(x_values)-100, max(x_values)+100)
    ax.set_ylim(min(y_values)-100, max(y_values)+100)

    ax.set_title(f"tag: {example_data['annotations'][i]['label']}")

fig.suptitle(f"Image ID: {example_image_id}")
fig.tight_layout()

plt.show()

In [ ]:
# BLUR IMAGE BY RESOLUTION

random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

fig, axs = plt.subplots(2, 2, figsize=(24, 8), tight_layout=True, constrained_layout=True)
axs = axs.flatten()
for i, ax in enumerate(axs):
    processed_image = resize(original_image, (resolutions_dict[50][0], resolutions_dict[50][0], 3), anti_aliasing=True)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    # processed_image = skimage.measure.block_reduce(original_image, (resolution_options[i],resolution_options[i],1), np.mean).astype(np.uint8)
    # processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    ax.imshow(processed_image)
    ax.set_title(f"Block size: {resolution_options[i]}")
    plt.show()


# plt.tight_layout()
plt.show()


In [ ]:
wide_image_id = example_image_data['wideImageID']
wide_im_path = env.download_image(int(wide_image_id))
wide_image = io.imread(wide_im_path)
wide_image_data = env.eti_api.get_images_data([wide_image_id])

wide_image_shape = wide_image.shape[0], wide_image.shape[1]
wide_image_shape

In [ ]:


fig, axs = plt.subplots(2, 2, figsize=(24, 8), tight_layout=True, constrained_layout=True)
axs = axs.flatten()
for i, ax in enumerate(axs):
    processed_image = resize(original_image, (resolutions_dict[50][0], resolutions_dict[50][0], 3), anti_aliasing=True)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    # processed_image = skimage.measure.block_reduce(original_image, (resolution_options[i],resolution_options[i],1), np.mean).astype(np.uint8)
    # processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)
    ax.imshow(processed_image)
    ax.set_title(f"Block size: {resolution_options[i]}")
    plt.show()


In [ ]:
# RESIZE IMAGES AND SAVE - FROM IMAGES SIZE LIST

resized_images_folder = os.path.join("images", "resized_images")
os.makedirs(resized_images_folder, exist_ok=True)

# random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
# random_image_id = 6415975
# random_image_data = images_data[images_data['imageID'] == random_image_id]

random_image_id = example_image_id

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

for i in range(len(ax_0)):
    processed_image_name = f"{random_image_id}_{i}_try4.jpg"
    processed_image_path = os.path.join(resized_images_folder, processed_image_name)
    processed_image = resize(original_image, (ax_1[i], ax_0[i], 3), anti_aliasing=True)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)

    io.imsave(processed_image_path, processed_image)
    processed_image = None


In [ ]:
# RESIZE IMAGES AND SAVE - FROM RESOLUTION DICT

resized_images_folder = os.path.join("images", "resized_images")
os.makedirs(resized_images_folder, exist_ok=True)

random_image_id = int(os.path.basename(np.random.choice(jsons_paths_list)).replace(".json", ""))
# random_image_id = 6415975
random_image_data = images_data[images_data['imageID'] == random_image_id]

im_path = env.download_image(random_image_id)
original_image = io.imread(im_path)

for i, height in enumerate(resolutions_dict.keys()):
    processed_image_name = f"{random_image_id}_h_{height}_try.jpg"
    processed_image_path = os.path.join(resized_images_folder, processed_image_name)
    processed_image = resize(original_image, (resolutions_dict[height][1], resolutions_dict[height][0], 3), anti_aliasing=True)
    processed_image = resize(processed_image, (original_image.shape[0], original_image.shape[1]), mode='constant', anti_aliasing=False)

    io.imsave(processed_image_path, processed_image)

In [ ]:
# num_steps = 5
# step_size = (wide_image_resolution - zoom_image_resolution) / (num_steps - 1)
# resolutions_list = np.arange(zoom_image_resolution, wide_image_resolution + 2*step_size, step_size)

# resolutions_list = resolution_options

In [ ]:
# CREATE RESOLUTION DICTIONARY BY ZOOM IMAGE SIZE

# GET WIDE IMAGE SHAPE
# example_image_id = 5246509
# example_image_id = 6415975
zoom_im_path = env.download_image(int(example_image_id))

zoom_image = io.imread(zoom_im_path)
wide_im_id = env.eti_api.get_matching_wide_images([example_image_id])[0]
wide_im_path = env.download_image(wide_im_id)
wide_image = io.imread(wide_im_path)
wide_image_shape = wide_image.shape[:-1]

# grid_shape = (wide_image_shape[1]//7, wide_image_shape[0]//8)
num_steps = 5

step_size_0 = (wide_image_shape[0] - wide_image_shape[0]//7) / (num_steps - 1)
step_size_1 = (wide_image_shape[1] - wide_image_shape[1]//8) / (num_steps - 1)

ax_0 = np.arange(wide_image_shape[0]//7, wide_image_shape[0], step_size_0).astype(int)
ax_1 = np.arange(wide_image_shape[1]//8, wide_image_shape[1], step_size_1).astype(int)

del zoom_image, wide_image

# Read dataloop annotations

In [ ]:
# DOWNLOAD ANNOTATIONS - POLYGONS

DATASET_NAME = ""
TASK_NAME = 'anafa_2023_06_23_resolution_lim_first_task_01'
VERSION = 0

dataloop_local_data_dir = os.path.join(DATA_DIR, f"dataloop")
annotation_local_path = os.path.join(dataloop_local_data_dir, f"annotations_{DATASET_NAME}_task_{TASK_NAME}_v{VERSION}")

jsons_folder = os.path.join(annotation_local_path, "json")
jsons_paths_list = glob.glob(os.path.join(jsons_folder, "*.json"))

polygons_task = project.tasks.get(task_name=TASK_NAME)
# task.


dataset = project.datasets.get(dataset_name=DATASET_NAME)
# # dataset.download(local_path=dataloop_local_data_dir,
# #                  annotation_options=dl.VIEW_ANNOTATION_OPTIONS_JSON)
dataset.download_annotations(local_path=annotation_local_path)

In [ ]:
# EXAMPLE - DISPLAY ONE IMAGE POINTS TAGS

example_image_id = 9638024
example_im_path = env.download_image(int(image_id))
example_image = io.imread(im_path)
original_image_width = example_image.shape[1]
original_image_height = example_image.shape[0]

# image_id = points_task_image_ids_list[11]
# image_id = 9638024
image_id = 9445268
points_tags_json = [path for path in points_jsons_paths_list if str(image_id) in path][0]
polygons_tags_json = [path for path in polygons_jsons_paths_list if str(image_id) in path][0]


with open(points_tags_json) as file:
    points_json_data = json.load(file)

im_path = env.download_image(int(image_id))
image = io.imread(im_path)


fig, ax = plt.subplots()
ax.imshow(image)


for i in range(len(points_json_data['annotations'])):
    width = points_json_data['metadata']['system']['width']
    height = points_json_data['metadata']['system']['height']

    width_factor = original_image_width / width
    height_factor = original_image_height / height

    x_value = int(points_json_data['annotations'][i]['coordinates']['x']) * width_factor
    y_value = int(points_json_data['annotations'][i]['coordinates']['y']) * height_factor

    # print(f"x: {points_json_data['annotations'][i]['coordinates']['x'], x_value}")
    # print(f"y: {points_json_data['annotations'][i]['coordinates']['y'], y_value}")
    
    ax.plot(x_value, y_value, 'mo')
    ax.set_xlim(1800, 2100)
    ax.set_ylim(2800, 2500)

    ax.set_title(f"image: {im_path}")

plt.show()

In [ ]:
# EXAMPLE - DISPLAY ONE IMAGE POLYGONS TAGS

# image_id = points_task_image_ids_list[9]

image_id = 9638024

points_tags_json = [path for path in points_jsons_paths_list if str(image_id) in path][0]
polygons_tags_json = [path for path in polygons_jsons_paths_list if str(image_id) in path][0]


with open(polygons_tags_json) as file:
    polygons_json_data = json.load(file)


im_path = env.download_image(int(image_id))
image = io.imread(im_path)


fig, ax = plt.subplots()
ax.imshow(image)

for i in range(len(polygons_json_data['annotations'])):

    x_values = [coord['x'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    y_values = [coord['y'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')

    ax.add_patch(polygon)

    # ax.set_xlim(0, image.shape[1])
    # ax.set_ylim(0, image.shape[0])
    # ax.set_ylim(image.shape[0], 0)
    # ax.set_xlim(0, 500)
    # ax.set_ylim(3500, 3800)


    ax.set_title(f"image: {im_path}")


plt.show()


# read_dataloop_annotations_resolution_5_res_20_images

In [ ]:
def create_polygons_df_example(polygons_list, image_id):

    im_path = env.download_image(int(image_id))
    image = io.imread(im_path)

    plt.figure()
    plt.imshow(image)

    polygons_df = pd.DataFrame()
    polygons_df['overlapping_polygons_indexes'] = find_overlapping_polygons_indexes(polygons_list)
    example_overlapping_polygons_indexes = polygons_df['overlapping_polygons_indexes'][0]
    example_overlapping_polygons = list(map(lambda i: polygons_list[i], example_overlapping_polygons_indexes))
    poly_intersection = get_polygons_intersection(example_overlapping_polygons)
    poly_union = get_polygons_union(example_overlapping_polygons)
    poly_iou = get_polygons_iou(example_overlapping_polygons)
    poly_box = get_polygons_box(poly_union, dilation = 0)

    p1_patch = PolygonPatch(np.array(poly_intersection.exterior.coords), facecolor='none', edgecolor='cornflowerblue', linewidth=2)
    p2_patch = PolygonPatch(np.array(poly_union.exterior.coords), facecolor='none', edgecolor='orange', linewidth=2)
    p4_patch = PolygonPatch(np.array(poly_box.exterior.coords), facecolor='none', edgecolor='springgreen', linewidth=2)


    minx, miny, maxx, maxy = poly_union.bounds

    # p3_patch = PolygonPatch(np.array(poly_iou.exterior.coords), facecolor='none', edgecolor='green', linewidth=2)
    # minx, miny, maxx, maxy = poly_box
    # width = maxx - minx
    # height = maxy - miny
    # p4_patch = Rectangle((minx, miny), width, height, facecolor='none', edgecolor='springgreen', linewidth=2)


    plt.gca().add_patch(p1_patch)
    plt.gca().add_patch(p2_patch)
    # plt.gca().add_patch(p3_patch)
    plt.gca().add_patch(p4_patch)

    text_x, text_y = minx, miny-50
    plt.text(text_x, text_y, "intersection", fontsize=12, color='cornflowerblue', ha='center', va='center')
    plt.text(text_x, text_y+20, "union", fontsize=12, color='orange', ha='center', va='center')
    # plt.text(text_x, text_y, "intersection", fontsize=12, color='cornflowerblue', ha='center', va='center')



    plt.xlim(minx-100, maxx+100)
    plt.ylim(maxy+100, miny-100)

    plt.savefig('images/output_image_with_polygons_8.jpg')
    plt.close()

In [ ]:
    # intersection_patch = PolygonPatch(np.array(poly_intersection.exterior.coords), facecolor='none', edgecolor='cornflowerblue', linewidth=2)
    # union_patch = PolygonPatch(np.array(poly_union.exterior.coords), facecolor='none', edgecolor='orange', linewidth=2)
    # box_patch = PolygonPatch(np.array(poly_box.exterior.coords), facecolor='none', edgecolor='springgreen', linewidth=2)

    # minx, miny, maxx, maxy = poly_union.bounds

    # plt.gca().add_patch(intersection_patch)
    # plt.gca().add_patch(union_patch)
    # plt.gca().add_patch(box_patch)

    # text_x, text_y = minx, miny-50
    # plt.text(text_x, text_y, "intersection", fontsize=12, color='cornflowerblue', ha='center', va='center')
    # plt.text(text_x, text_y+20, "union", fontsize=12, color='orange', ha='center', va='center')
    # plt.text(f"Image {image_id}, IOU: {polygons_df['poly_iou']}")

    # # plt.xlim(minx-100, maxx+100)
    # # plt.ylim(maxy+100, miny-100)

    # image_with_polygons_path = f'images/output_image_with_polygons_{image_with_polygons_path}_8.jpg'
    # plt.savefig(image_with_polygons_path)
    # plt.close()
    # print(f"Saved image {image_with_polygons_path}.")


In [ ]:
def get_polygons_intersection(overlapping_polygons_list):
    overlapping_polygons_list = reduce_contained_polygons(overlapping_polygons_list)

    intersection_polygon = reduce(lambda x, y: x.intersection(y), overlapping_polygons_list)

    # intersection_polygons_list = []
    # while True:
    #     if len(intersection_polygons_list) < 1:
    #         intersection_polygons_list = overlapping_polygons_list.copy()
    #         break
    #     elif len(intersection_polygons_list) == 1:
    #         break
    #     else:
    #         overlapping_polygons_list = intersection_polygons_list.copy()
    #         intersection_polygons_list = []
    #     for i in range(1, len(overlapping_polygons_list)):
    #         intersection_polygons_list.append(overlapping_polygons_list[0].intersection(overlapping_polygons_list[i]))

    return intersection_polygon




    # for i in range(len(overlapping_polygons_list)):
    #     for j in range(i+1, len(overlapping_polygons_list)):
    #         intersection_polygons_list.append(overlapping_polygons_list[i].intersection(overlapping_polygons_list[j]))
    # if len(intersection_polygons_list)>1:
    #     return get_polygons_intersection(intersection_polygons_list)
    # else:
    #     return intersection_polygons_list[0]


def get_polygons_union(overlapping_polygons_list):
    union_polygon = reduce(lambda x, y: x.union(y), overlapping_polygons_list)
    return union_polygon



    # union_polygons_list = []
    # for i in range(len(overlapping_polygons_list)):
    #     for j in range(i+1, len(overlapping_polygons_list)):
    #         union_polygons_list.append(overlapping_polygons_list[i].union(overlapping_polygons_list[j]))
    # if len(union_polygons_list)>1:
    #     return get_polygons_intersection(union_polygons_list)
    # else:
    #     return union_polygons_list[0]

# Resolution limitation exploration

In [ ]:
# EXAMPLE - DISPLAY ONE IMAGE TAGS - ONE POINT

example_image_id = 5233797
example_im_path = env.download_image(int(example_image_id))
example_image = io.imread(example_im_path)
original_image_width = example_image.shape[1]
original_image_height = example_image.shape[0]

# image_id = points_task_image_ids_list[15] 
image_id = 5852565
# image_id = 6580458
# image_id = 9445268
points_tags_json = [path for path in points_jsons_paths_list if str(image_id) in path][0]
polygons_tags_json = [path for path in polygons_jsons_paths_list if str(image_id) in path][0]


im_path = env.download_image(int(image_id))
image = io.imread(im_path)


with open(points_tags_json) as file:
    points_json_data = json.load(file)

with open(polygons_tags_json) as file:
    polygons_json_data = json.load(file)



fig, ax = plt.subplots(figsize=(20,10))

ax.imshow(image)

shapely_polygons = []

# DRAW POLYGONS
for i in range(len(polygons_json_data['annotations'])):
    x_values = [coord['x'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    y_values = [coord['y'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')
    shapely_polygons.append(Polygon(list(zip(x_values, y_values))))
    ax.add_patch(polygon)
    ax.set_title(f"image: {im_path}")


# DRAW POINTS
for i in range(len(points_json_data['annotations'])):
    width = points_json_data['metadata']['system']['width']
    height = points_json_data['metadata']['system']['height']

    width_factor = original_image_width / width
    height_factor = original_image_height / height

    x_value = int(points_json_data['annotations'][i]['coordinates']['x']) * width_factor
    y_value = int(points_json_data['annotations'][i]['coordinates']['y']) * height_factor

    shapely_point = Point(x_value, y_value)
    print(shapely_point)

    is_inside = np.any([poly.contains(shapely_point) for poly in shapely_polygons])
    if is_inside:
        point_color = 'mo'
    else:
        point_color = 'ro'

    ax.plot(x_value, y_value, point_color)

    ax.set_xlim(0, 400)
    ax.set_ylim(3750, 3500)
    # ax.set_xlim(1800, 2100)
    # ax.set_ylim(2800, 2500)


    ax.set_title(f"image: {im_path}. {len(points_json_data['annotations'])} point tags")


plt.show()

# Read DL annotations infestation

In [ ]:
def fit_points_to_boxes_old(boxes, labels):
    for i, box_row in boxes.iterrows():
        pt_list = []
        for j, label_row in labels.iterrows():
            x_value = int(label_row['coordinates']['x'])
            y_value = int(label_row['coordinates']['y'])
            shapely_point = Point(x_value, y_value)
            is_inside = box_row['box_poly'].contains(shapely_point)
            if is_inside:
                pt_list.append(shapely_point)
        boxes.at[i, "fitting_points"] = str(pt_list)
    return boxes

In [ ]:
points_indexes = box_df['fitting_points_indexes']
# points_indexes = boxes['fitting_points_indexes'][0]
box_labels = labels.iloc[points_indexes]
most_common_label = labels.iloc[points_indexes]['label'].value_counts().idxmax()
box_labels[box_labels['label'].str.contains(r'Infestation|No')]

In [ ]:
labels.iloc[infestation_rows]['label'].value_counts().max()

In [ ]:
def get_infestation_avg(box_labels):
     infestation_dict = {
         f"Infestation.Up_to_10%": 5,
         f"Infestation.10%-25%": 17.5,
         f"Infestation.25%-50%": 37.5,
         f"Infestation.50%-75%": 65.5,
         f"Infestation.above_75%": 87.5
     }
     labels_numbers = box_labels.replace(infestation_dict)
     return labels_numbers.mean()

def get_box_label_and_infestation_avg(box_df, labels):
    points_indexes = box_df['fitting_points_indexes']
    box_labels = labels.iloc[points_indexes]

    infestation_rows = box_labels[box_labels['label'].str.contains("Infestation")].index
    no_weeds_rows = box_labels[box_labels['label'].str.contains("No")].index

    box_final_label = labels.iloc[infestation_rows]['label'].value_counts().idxmax()
    votes = labels.iloc[infestation_rows]['label'].value_counts().max()
    
    if len(infestation_rows) > 0:
        infestation_avarage = get_infestation_avg(labels.iloc[infestation_rows]['label'])
    else:
        infestation_avarage = 0
    return box_final_label, infestation_avarage, votes

boxes["box_final_label"] = None
boxes["infestation_avarage"] = None
for i, box_row in boxes.iterrows():
    box_final_label, infestation_avarage, votes = get_box_label_and_infestation_avg(box_row, labels)
    boxes.at[i, "box_final_label"] = box_final_label
    boxes.at[i, "infestation_avarage"] = infestation_avarage
    boxes.at[i, "votes"] = votes

In [ ]:
def get_infestation_avg(box_labels):
     infestation_dict = {
         f"Infestation.Up_to_10%": 5,
         f"Infestation.10%-25%": 17.5,
         f"Infestation.25%-50%": 37.5,
         f"Infestation.50%-75%": 65.5,
         f"Infestation.above_75%": 87.5
     }
     labels_numbers = box_labels.replace(infestation_dict)
     return labels_numbers.mean()

def get_box_label_and_infestation_avg(box_df, labels):
    points_indexes = box_df['fitting_points_indexes']
    box_labels = labels.iloc[points_indexes]

    infestation_rows = box_labels[box_labels['label'].str.contains("Infestation")].index
    no_weeds_rows = box_labels[box_labels['label'].str.contains("No")].index

    if len(no_weeds_rows) > len(infestation_rows):
        box_final_label = labels.iloc[infestation_rows]['label'].value_counts().idxmax()
        votes = labels.iloc[infestation_rows]['label'].value_counts().max()
        if len(infestation_rows) > 0:
            infestation_avarage = get_infestation_avg(labels.iloc[infestation_rows]['label'])
        else:
            infestation_avarage = 0
    elif len(infestation_rows) > 0:
        box_final_label = labels.iloc[infestation_rows]['label'].value_counts().idxmax()
        print(f"infestation label: {box_final_label}")
        infestation_avarage = get_infestation_avg(labels.iloc[infestation_rows]['label'])
        votes = labels.iloc[infestation_rows]['label'].value_counts().max()
    else:
        raise Exception(f"Problem whit labels in image id: {im_id}")

    return box_final_label, infestation_avarage, votes

boxes["box_final_label"] = None
boxes["infestation_avarage"] = None
for i, box_row in boxes.iterrows():
    box_final_label, infestation_avarage = get_box_label_and_infestation_avg(box_row, labels)
    boxes.at[i, "box_final_label"] = box_final_label
    boxes.at[i, "infestation_avarage"] = infestation_avarage

In [ ]:
def get_box_label_and_infestation_avg(box_df, labels):
    points_indexes = box_df['fitting_points_indexes']
    box_labels = labels.iloc[points_indexes]

    infestation_rows = box_labels[box_labels['label'].str.contains("Infestation")].index
    no_weeds_rows = box_labels[box_labels['label'].str.contains("No")].index

    if len(infestation_rows) > 0:
        infestation_avarage = get_infestation_avg(labels.iloc[infestation_rows]['label'])
        box_final_label = labels.iloc[infestation_rows]['label'].value_counts().idxmax()
        votes = labels.iloc[infestation_rows]['label'].value_counts().max()
    else:
        infestation_avarage = 0
        box_final_label = labels.iloc[no_weeds_rows]['label'].value_counts().idxmax()
        votes = labels.iloc[no_weeds_rows]['label'].value_counts().max()
    return box_final_label, infestation_avarage, votes

In [ ]:
"args": [
    // "--order_ids=[405497, 404108]",
    "--wide_image_ids=[9134832, 8722161]",
    // "--wide_image_ids=[8484421, 7876108, 4956894, 6859096, 7014238, 7490734, 7712145, 8722161, 9134832]",
    // "--wide_image_ids=[8722161, 9134832]",
    // "--zom_image_ids=[8471755, 7875316]",

In [ ]:
# def choose_canopy_map(index_canopy_map, hsv_canopy_map):
#     hsv_canopy_percent, canopy_index_percent = calculate_canopy_percentage(index_canopy_map, hsv_canopy_map)

#     return


# def save_histograms(hsv_canopy_percent_list, canopy_index_percent_list, dir="data"):
#     data_lists = {"hsv_percent_list": hsv_canopy_percent_list,
#                   "index_percent_list": canopy_index_percent_list}

#     fig, axes = plt.subplots(1, 2, figsize=(10, 4))

#     for i in range(len(data_lists)):
#         data = data_lists[list(data_lists.keys())[i]]
#         axes[i].hist(data, bins=20, color='lightseagreen', alpha=0.7)
#         axes[i].set_xlabel('Canopy coverage percent')
#         axes[i].set_ylabel('Frequency')
#         axes[i].set_title(list(data_lists.keys())[i].replace("_list", ""))
#     plt.tight_layout()
#     current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#     plt.savefig(os.path.join(dir, f'side_by_side_histograms_{current_datetime}.jpg'))
#     print('h')